In [1]:
!pip install transformers
!pip install torch
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 16.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 26.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Step 2: Prepare the dataset
# Assuming your dataset has 'text' and 'label' columns in a CSV file
df = pd.read_csv('/kaggle/input/arabichatespeech/hatespeech.csv')

# Step 3: Tokenize the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.texts = dataframe['sentence']
        self.labels = dataframe['type']
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = int(self.labels[index])

        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Step 4: Create DataLoader
dataset = CustomDataset(df, tokenizer, max_length=128)
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Step 5: Define the model and training loop
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Batch Loss: {loss.item():.4f}")
        
model.save_pretrained('modelWeights')

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Batch Loss: 0.6979
Epoch 1/3, Batch Loss: 0.5960
Epoch 1/3, Batch Loss: 0.6277
Epoch 1/3, Batch Loss: 0.5196
Epoch 1/3, Batch Loss: 0.6539
Epoch 1/3, Batch Loss: 0.4139
Epoch 1/3, Batch Loss: 0.6203
Epoch 1/3, Batch Loss: 0.7025
Epoch 1/3, Batch Loss: 0.4105
Epoch 1/3, Batch Loss: 0.5961
Epoch 1/3, Batch Loss: 0.3548
Epoch 1/3, Batch Loss: 0.4725
Epoch 1/3, Batch Loss: 0.5161
Epoch 1/3, Batch Loss: 0.4804
Epoch 1/3, Batch Loss: 0.2737
Epoch 1/3, Batch Loss: 0.3013
Epoch 1/3, Batch Loss: 0.5024
Epoch 1/3, Batch Loss: 0.7031
Epoch 1/3, Batch Loss: 0.2694
Epoch 1/3, Batch Loss: 0.4839
Epoch 1/3, Batch Loss: 0.3585
Epoch 1/3, Batch Loss: 0.6107
Epoch 1/3, Batch Loss: 0.3429
Epoch 1/3, Batch Loss: 0.2123
Epoch 1/3, Batch Loss: 0.7340
Epoch 1/3, Batch Loss: 0.5540
Epoch 1/3, Batch Loss: 0.7447
Epoch 1/3, Batch Loss: 0.8450
Epoch 1/3, Batch Loss: 0.3993
Epoch 1/3, Batch Loss: 0.5741
Epoch 1/3, Batch Loss: 0.5286
Epoch 1/3, Batch Loss: 0.4700
Epoch 1/3, Batch Loss: 0.6511
Epoch 1/3,

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the trained model and tokenizer
model_path = 'path_to_save_model'  # Replace with the path to your saved model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(model_path)

# Function to classify new text
def classify_text(text):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

        # Assuming your model is binary classification (0 or 1)
        if predicted_label == 1:
            return "Hate Speech"
        else:
            return "Not Hate Speech"

OSError: path_to_save_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:

new_text = "لا"
predicted_label = classify_text(new_text)
print("Predicted Label:", predicted_label)